## URL matching

**Purpose**: check each outlet's document to see if it was (re-)tweeted, and if so, how many times.

1. Update `standardized_url` field to use the updated version of urlExpander
2. Add matches: for each outlet's document, check if `standardized_url` matches 1+ documents of `tweets2_url` doctype.

In [1]:
# matplotlib is logged even though disable_existing_loggers=yes in logging_config.yaml
# https://stackoverflow.com/a/51529172/7016397
# workaround is to manually set the level before creating my logger
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from usrightmedia.shared.loggers import get_logger
LOGGER = get_logger(filename = '01-url-matching', logger_type='main')

- Pre-processing uses the Bulk API through the Python Elasticsearch client.
- Elasticsearch: ["How Big Is Too Big?"](https://www.elastic.co/guide/en/elasticsearch/guide/current/bulk.html#_how_big_is_too_big)
>The entire bulk request needs to be loaded into memory by the node that receives our request, so the bigger the request, the less memory available for other requests. There is an optimal size of bulk request. Above that size, performance no longer improves and may even drop off. The optimal size, however, is not a fixed number. It depends entirely on your hardware, your document size and complexity, and your indexing and search load.
>Fortunately, it is easy to find this sweet spot: Try indexing typical documents in batches of increasing size. When performance starts to drop off, your batch size is too big. A good place to start is with batches of 1,000 to 5,000 documents or, if your documents are very large, with even smaller batches.
>It is often useful to keep an eye on the physical size of your bulk requests. One thousand 1KB documents is very different from one thousand 1MB documents. A good bulk size to start playing with is around 5-15MB in size.

In [2]:
bulksize = 2000

## Import INCA and check doctypes

In [3]:
from inca import Inca
myinca = Inca()
myinca.database.list_doctypes()

2021-12-16 18:39:41,657 - [WARNING] - INCA - (hype_analysis.py).<module>(21) - $DISPLAY environment variable is not set, trying a different approach. You probably are running INCA on a text console, right?
2021-12-16 18:39:41,920 - [WARNING] - INCA - (var_tsa_analysis.py).<module>(21) - $DISPLAY environment variable is not set, trying a different approach. You probably are running INCA on a text console, right?


{'tweets2': 889739,
 'tweets2_url': 285447,
 'foxnews': 264620,
 'breitbart': 149241,
 'dailycaller': 121822,
 'oneamericanews': 117287,
 'washingtonexaminer': 71710,
 'newsmax': 71146,
 'gatewaypundit': 39401,
 'infowars': 28453,
 'vdare': 19643,
 'dailystormer': 15823,
 'americanrenaissance': 9838,
 'rushlimbaugh': 9396,
 'seanhannity': 5647}

### 1. Add `standardized_url` field to documents representing (re-)tweeted URLs and outlets' articles
- `force=True` to use updated version of urlExpander (public fork)

In [4]:
doctypes_to_process = [
    "americanrenaissance",
    "breitbart",
    "dailycaller",
    "dailystormer",
    "foxnews",
    "gatewaypundit",
    "infowars",
    "newsmax",
    "oneamericanews",
    "rushlimbaugh",
    "seanhannity",
    "vdare",
    "washingtonexaminer",
    "tweets2_url"
]

In [5]:
for doctype in doctypes_to_process:
    
    try:
        
        docs = myinca.processing.standardize_url(docs_or_query=doctype,
                                                 field="resolved_url",
                                                 save=True,
                                                 new_key="standardized_url",
                                                 action="batch",
                                                 bulksize=bulksize,
                                                 force=True)
        for doc in docs:
            # runs process on doc
            pass

    except Exception as e:
        LOGGER.warning(e)

100%|██████████| 285447/285447 [36:55<00:00, 128.83it/s]


### 2. Find matches
- For each outlet's document, check if `standardized_url` matches any documents of the `tweets2_url` doctype.
- Store the results in the new keys: `tweets2_url_ids` (list of matched IDs), `tweets2_url_count` (number of matches), and `tweets2_url_ind` (boolean indicator of 1+ matches).

In [6]:
outlet_doctypes = [
    "americanrenaissance",
    "breitbart",
    "dailycaller",
    "dailystormer",
    "foxnews",
    "gatewaypundit",
    "infowars",
    "newsmax",
    "oneamericanews",
    "rushlimbaugh",
    "seanhannity",
    "vdare",
    "washingtonexaminer",
]

In [7]:
# action="run" rather than action="batch" because this processor sends an individual HTTP request to Elasticsearch per document anyway
for doctype in outlet_doctypes:
    
    try:
        
        docs = myinca.processing.match_outlet_articles_to_tweets2_urls(docs_or_query=doctype,
                                                                       field="standardized_url",
                                                                       save=True,
                                                                       new_key="tweets2_url_ids",
                                                                       action="run",
                                                                       force=True)
        for doc in docs:
            # runs process on doc
            pass

    except Exception as e:
        LOGGER.warning(e)

100%|██████████| 71710/71710 [20:42<00:00, 57.73it/s]


In [8]:
for doctype in outlet_doctypes:
    
    try:
        
        docs = myinca.processing.match_outlet_articles_to_tweets2_urls_count(docs_or_query=doctype,
                                                                             field="tweets2_url_ids",
                                                                             save=True,
                                                                             new_key="tweets2_url_match_count",
                                                                             action="batch",
                                                                             bulksize=bulksize,
                                                                             force=True)
        for doc in docs:
            # runs process on doc
            pass

    except Exception as e:
        LOGGER.warning(e)

100%|██████████| 71710/71710 [19:21<00:00, 61.73it/s] 


In [9]:
for doctype in outlet_doctypes:
    
    try:
        
        docs = myinca.processing.match_outlet_articles_to_tweets2_urls_ind(docs_or_query=doctype,
                                                                           field="tweets2_url_ids",
                                                                           save=True,
                                                                           new_key="tweets2_url_match_ind",
                                                                           action="batch",
                                                                           bulksize=bulksize,
                                                                           force=True)
        for doc in docs:
            # runs process on doc
            pass

    except Exception as e:
        LOGGER.warning(e)

100%|██████████| 71710/71710 [20:36<00:00, 57.99it/s] 
